# Task 1 of Assignment 2

**Objective:** Process the MIMIC chest X-ray dataset as a single entity, training all pathologies together.

**Split Ratio:** 5:3:2 (50% training, 30% validation, 20% testing)

**Pathology Grouping:** No grouping, all 13 pathologies (Atelectasis, Cardiomegaly, etc.) treated uniformly

**Key Features:**
- Data split with fixed seed (seed=42) for consistency
- NaN values converted to 0, no filtering of all-NaN samples
- Outputs three datasets: train_dataset, val_dataset, test_dataset

### 1. Load Data (All Label)

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
from tqdm import tqdm
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Set TensorFlow logging level to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

# Helper function: Load embeddings from TFRecord files
def load_embedding(embedding_path):
    raw_dataset = tf.data.TFRecordDataset([embedding_path])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value
    embedding = torch.tensor(embedding_values)
    if torch.isnan(embedding).any() or torch.isinf(embedding).any():
        raise ValueError(f"NaN or Inf found in embedding: {embedding_path}")
    return embedding

# Custom dataset class for MIMIC chest X-ray embeddings
class MIMIC_Embed_Dataset(Dataset):
    pathologies = [
        "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", "Lung Lesion",
        "Edema", "Consolidation", "Pneumonia", "Atelectasis", "Pneumothorax",
        "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"
    ]

    def __init__(self, embedpath, csvpath, metacsvpath, views=["PA"], mode="train", seed=42, split_ratio=[0.5, 0.3, 0.2], unique_patients=True):
        super().__init__()
        np.random.seed(seed)  # Set seed for reproducibility
        self.pathologies = sorted(self.pathologies)
        self.mode = mode
        self.embedpath = embedpath
        self.csv = pd.read_csv(csvpath)
        self.metacsv = pd.read_csv(metacsvpath)
        self.split_ratio = split_ratio

        # Merge CSV data
        self.csv = self.csv.set_index(["subject_id", "study_id"])
        self.metacsv = self.metacsv.set_index(["subject_id", "study_id"])
        self.csv = self.csv.join(self.metacsv).reset_index()

        self.csv["view"] = self.csv["ViewPosition"]
        self.limit_to_selected_views(views)

        if unique_patients:
            self.csv = self.csv.groupby("subject_id").first().reset_index()

        # Split dataset into train, validation, and test sets
        n_row = self.csv.shape[0]
        if self.mode == "train":
            self.csv = self.csv[:int(n_row * self.split_ratio[0])]
        elif self.mode == "valid":
            self.csv = self.csv[int(n_row * self.split_ratio[0]):int(n_row * (self.split_ratio[0] + self.split_ratio[1]))]
        elif self.mode == "test":
            self.csv = self.csv[int(n_row * (self.split_ratio[0] + self.split_ratio[1])):]
        else:
            raise ValueError(f"mode must be one of [train, valid, test], got {self.mode}")

        # Process labels
        healthy = self.csv["No Finding"] == 1
        labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                self.csv.loc[healthy, pathology] = 0
                mask = self.csv[pathology]
            labels.append(mask.values)
        self.labels = np.asarray(labels).T.astype(np.float32)
        self.labels[np.isnan(self.labels)] = 0  # Replace all NaN values with 0
        self.labels = np.where((self.labels != 0) & (self.labels != 1), 0, self.labels)

        self.csv = self.csv.reset_index(drop=True)
        self.pathologies = list(np.char.replace(self.pathologies, "Pleural Effusion", "Effusion"))
        self.csv["offset_day_int"] = self.csv["StudyDate"]
        self.csv["patientid"] = self.csv["subject_id"].astype(str)

    def limit_to_selected_views(self, views):
        # Limit dataset to specified views
        if not isinstance(views, list):
            views = [views]
        if '*' in views:
            views = ["*"]
        self.views = views
        self.csv["view"] = self.csv["view"].fillna("UNKNOWN")
        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Retrieve a single sample from the dataset
        sample = {}
        sample["idx"] = idx
        sample["lab"] = self.labels[idx]
        subjectid = str(self.csv.iloc[idx]["subject_id"])
        studyid = str(self.csv.iloc[idx]["study_id"])
        dicom_id = str(self.csv.iloc[idx]["dicom_id"])
        embed_file = os.path.join(self.embedpath, f"p{subjectid[:2]}", f"p{subjectid}", f"s{studyid}", f"{dicom_id}.tfrecord")
        sample["embedding"] = load_embedding(embed_file)
        return sample

# Function to initialize datasets with given paths, seed, and split ratio
def initialize_mimic_datasets(embedpath, csvpath, metacsvpath, seed=42, split_ratio=[0.5, 0.3, 0.2]):
    """
    Initialize MIMIC chest X-ray datasets with specified paths, seed, and split ratio.
    
    Args:
        embedpath (str): Path to the embedding files
        csvpath (str): Path to the CheXpert CSV file
        metacsvpath (str): Path to the metadata CSV file
        seed (int): Random seed for reproducibility (default: 42)
        split_ratio (list): Train, validation, test split ratio (default: [0.5, 0.3, 0.2])
    
    Returns:
        tuple: (train_dataset, val_dataset, test_dataset)
    """
    train_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="train",
        seed=seed,
        split_ratio=split_ratio
    )
    val_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="valid",
        seed=seed,
        split_ratio=split_ratio
    )
    test_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="test",
        seed=seed,
        split_ratio=split_ratio
    )
    return train_dataset, val_dataset, test_dataset

# Data paths
embedpath = "/Users/prestonlim/Documents/phd/sph6004/assignment2/data/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "/Users/prestonlim/Documents/phd/sph6004/assignment2/data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "/Users/prestonlim/Documents/phd/sph6004/assignment2/data/mimic-cxr-2.0.0-metadata.csv"

train_dataset, val_dataset, test_dataset = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

In [2]:
train_dataset[1000]

{'idx': 1000,
 'lab': array([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.], dtype=float32),
 'embedding': tensor([-0.6009, -2.6448,  1.1589,  ...,  0.3574,  1.4271, -2.1083])}

In [3]:
X_train = np.stack([sample['embedding'].numpy() for sample in train_dataset])  # shape: [n_samples, embedding_dim]
y_train = np.stack([sample['lab'] for sample in train_dataset])               # shape: [n_samples, n_labels]

In [4]:
X_validation = np.stack([sample['embedding'].numpy() for sample in val_dataset])
y_validation = np.stack([sample['lab'] for sample in val_dataset])

In [9]:
X_test = np.stack([sample['embedding'].numpy() for sample in test_dataset])
y_test = np.stack([sample['lab'] for sample in test_dataset])

In [5]:
X_train.shape

(22814, 1376)

In [6]:
y_train.shape

(22814, 13)

In [7]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, multilabel_confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
import joblib

In [18]:
assignment_dir = "/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models"
vanilla_logreg_model_path = f"{assignment_dir}/vanilla_logreg_model.pkl"
l1_logreg_model_path = f"{assignment_dir}/l1_logreg_model.pkl"
l2_logreg_model_path = f"{assignment_dir}/l2_logreg_model.pkl"
elasticnet_logreg_model_path = f"{assignment_dir}/elasticnet_logreg_model.pkl"

In [10]:
vanilla_logreg_model = MultiOutputClassifier(
    LogisticRegression(
        penalty=None,
        solver='lbfgs',
        max_iter=10000,
        n_jobs=-1,
        random_state=10
    )
).fit(X_train, y_train)

In [11]:
joblib.dump(vanilla_logreg_model, vanilla_logreg_model_path)

['/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models/vanilla_logreg_model.pkl']

In [12]:
y_pred_logreg = vanilla_logreg_model.predict(X_validation)
print("Logistic Regression:")
print(classification_report(y_validation, y_pred_logreg, zero_division=0, target_names=val_dataset.pathologies))

Logistic Regression:
                            precision    recall  f1-score   support

               Atelectasis       0.41      0.24      0.30      1211
              Cardiomegaly       0.40      0.26      0.31      1085
             Consolidation       0.06      0.14      0.08       243
                     Edema       0.16      0.27      0.20       356
Enlarged Cardiomediastinum       0.03      0.06      0.04       189
                  Fracture       0.07      0.06      0.06       292
               Lung Lesion       0.18      0.11      0.14       392
              Lung Opacity       0.42      0.23      0.30      1612
                  Effusion       0.55      0.45      0.49      1121
             Pleural Other       0.07      0.14      0.09       123
                 Pneumonia       0.25      0.10      0.14       685
              Pneumothorax       0.12      0.18      0.14       175
           Support Devices       0.28      0.23      0.26       572

                 micro av

In [13]:
l1_logreg_model = MultiOutputClassifier(LogisticRegressionCV(
    penalty='l1',
    solver='liblinear',
    Cs=10,
    max_iter=10000,
    n_jobs=-1,
    cv=5
))
l1_logreg_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegressionCV(cv=5, max_iter=10000,
                                                     n_jobs=-1, penalty='l1',
                                                     solver='liblinear'))

In [14]:
joblib.dump(l1_logreg_model, l1_logreg_model_path)

['/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models/l1_logreg_model.pkl']

In [15]:
y_pred_l1_logreg = l1_logreg_model.predict(X_validation)
print("L1 regularized Logistic Regression:")
print(classification_report(y_validation, y_pred_l1_logreg, zero_division=0, target_names=val_dataset.pathologies))

L1 regularized Logistic Regression:
                            precision    recall  f1-score   support

               Atelectasis       0.53      0.16      0.24      1211
              Cardiomegaly       0.53      0.18      0.27      1085
             Consolidation       0.00      0.00      0.00       243
                     Edema       0.65      0.11      0.19       356
Enlarged Cardiomediastinum       0.00      0.00      0.00       189
                  Fracture       0.00      0.00      0.00       292
               Lung Lesion       0.00      0.00      0.00       392
              Lung Opacity       0.51      0.13      0.21      1612
                  Effusion       0.68      0.45      0.54      1121
             Pleural Other       0.00      0.00      0.00       123
                 Pneumonia       0.68      0.03      0.05       685
              Pneumothorax       0.00      0.00      0.00       175
           Support Devices       0.47      0.08      0.13       572

          

In [16]:
l2_logreg_model = MultiOutputClassifier(LogisticRegressionCV(
    penalty='l2',
    solver='liblinear',
    Cs=10,
    max_iter=10000,
    n_jobs=-1,
    cv=5
))
l2_logreg_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegressionCV(cv=5, max_iter=10000,
                                                     n_jobs=-1,
                                                     solver='liblinear'))

In [17]:
joblib.dump(l2_logreg_model, l2_logreg_model_path)

['/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models/l2_logreg_model.pkl']

In [18]:
y_pred_l2_logreg = l2_logreg_model.predict(X_validation)
print("L2 regularized Logistic Regression:")
print(classification_report(y_validation, y_pred_l2_logreg, zero_division=0, target_names=val_dataset.pathologies))

L2 regularized Logistic Regression:
                            precision    recall  f1-score   support

               Atelectasis       0.50      0.17      0.25      1211
              Cardiomegaly       0.66      0.07      0.13      1085
             Consolidation       0.00      0.00      0.00       243
                     Edema       0.59      0.15      0.23       356
Enlarged Cardiomediastinum       0.00      0.00      0.00       189
                  Fracture       0.00      0.00      0.00       292
               Lung Lesion       0.00      0.00      0.00       392
              Lung Opacity       0.52      0.12      0.20      1612
                  Effusion       0.68      0.47      0.56      1121
             Pleural Other       0.00      0.00      0.00       123
                 Pneumonia       0.75      0.02      0.04       685
              Pneumothorax       0.00      0.00      0.00       175
           Support Devices       0.43      0.11      0.18       572

          

In [19]:
elasticnetratios = [0.3, 0.5, 0.7]
elasticnet_logreg_model = MultiOutputClassifier(LogisticRegressionCV(
    penalty='elasticnet',
    solver='saga',  # Elastic Net requires 'saga' solver
    Cs=10,  # Number of C values to try
    l1_ratios=elasticnetratios,
    max_iter=10000,
    n_jobs=-1,
    cv=5
))
elasticnet_logreg_model.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegressionCV(cv=5,
                                                     l1_ratios=[0.3, 0.5, 0.7],
                                                     max_iter=10000, n_jobs=-1,
                                                     penalty='elasticnet',
                                                     solver='saga'))

In [20]:
joblib.dump(elasticnet_logreg_model, elasticnet_logreg_model_path)

['/Users/prestonlim/Documents/phd/sph6004/assignment2/latest_models/elasticnet_logreg_model.pkl']

In [21]:
print("Elasticnet regularized Logistic Regression:")
y_pred_elasticnet_logreg = elasticnet_logreg_model.predict(X_validation)
print(classification_report(y_validation, y_pred_elasticnet_logreg, zero_division=0, target_names=val_dataset.pathologies))

Elasticnet regularized Logistic Regression:
                            precision    recall  f1-score   support

               Atelectasis       0.52      0.16      0.25      1211
              Cardiomegaly       0.54      0.19      0.28      1085
             Consolidation       0.00      0.00      0.00       243
                     Edema       0.55      0.14      0.23       356
Enlarged Cardiomediastinum       0.00      0.00      0.00       189
                  Fracture       0.80      0.01      0.03       292
               Lung Lesion       0.00      0.00      0.00       392
              Lung Opacity       0.53      0.09      0.15      1612
                  Effusion       0.67      0.45      0.54      1121
             Pleural Other       0.00      0.00      0.00       123
                 Pneumonia       0.64      0.04      0.08       685
              Pneumothorax       0.00      0.00      0.00       175
           Support Devices       0.51      0.04      0.08       572

  

### Load models

In [19]:
vanilla_logreg_model = joblib.load(vanilla_logreg_model_path)
l1_logreg_model = joblib.load(l1_logreg_model_path)
l2_logreg_model = joblib.load(l2_logreg_model_path)
elasticnet_logreg_model = joblib.load(elasticnet_logreg_model_path)

In [20]:
y_pred_logreg_test = vanilla_logreg_model.predict(X_test)
print("Logistic Regression test:")
print(classification_report(y_test, y_pred_logreg_test, zero_division=0, target_names=val_dataset.pathologies))

Logistic Regression test:
                            precision    recall  f1-score   support

               Atelectasis       0.42      0.25      0.31       855
              Cardiomegaly       0.42      0.26      0.32       761
             Consolidation       0.07      0.17      0.10       166
                     Edema       0.16      0.26      0.20       229
Enlarged Cardiomediastinum       0.05      0.09      0.07       139
                  Fracture       0.07      0.06      0.06       184
               Lung Lesion       0.16      0.10      0.12       271
              Lung Opacity       0.39      0.20      0.27      1120
                  Effusion       0.55      0.47      0.51       742
             Pleural Other       0.09      0.15      0.11        81
                 Pneumonia       0.28      0.12      0.17       431
              Pneumothorax       0.11      0.16      0.13       105
           Support Devices       0.30      0.24      0.26       398

                 mic

In [21]:
y_pred_l1_logreg_test = l1_logreg_model.predict(X_test)
print("L1 Logistic Regression test:")
print(classification_report(y_test, y_pred_l1_logreg_test, zero_division=0, target_names=val_dataset.pathologies))

L1 Logistic Regression test:
                            precision    recall  f1-score   support

               Atelectasis       0.52      0.14      0.22       855
              Cardiomegaly       0.61      0.20      0.30       761
             Consolidation       0.00      0.00      0.00       166
                     Edema       0.48      0.09      0.15       229
Enlarged Cardiomediastinum       0.00      0.00      0.00       139
                  Fracture       0.00      0.00      0.00       184
               Lung Lesion       0.00      0.00      0.00       271
              Lung Opacity       0.52      0.12      0.19      1120
                  Effusion       0.66      0.50      0.57       742
             Pleural Other       0.00      0.00      0.00        81
                 Pneumonia       0.81      0.05      0.09       431
              Pneumothorax       0.00      0.00      0.00       105
           Support Devices       0.42      0.08      0.13       398

                 

In [22]:
y_pred_l2_logreg_test = l2_logreg_model.predict(X_test)
print("L2 Logistic Regression test:")
print(classification_report(y_test, y_pred_l2_logreg_test, zero_division=0, target_names=val_dataset.pathologies))

L2 Logistic Regression test:
                            precision    recall  f1-score   support

               Atelectasis       0.51      0.17      0.25       855
              Cardiomegaly       0.67      0.08      0.15       761
             Consolidation       0.00      0.00      0.00       166
                     Edema       0.54      0.13      0.20       229
Enlarged Cardiomediastinum       0.00      0.00      0.00       139
                  Fracture       0.00      0.00      0.00       184
               Lung Lesion       0.00      0.00      0.00       271
              Lung Opacity       0.55      0.11      0.19      1120
                  Effusion       0.66      0.51      0.57       742
             Pleural Other       0.00      0.00      0.00        81
                 Pneumonia       0.71      0.03      0.05       431
              Pneumothorax       0.00      0.00      0.00       105
           Support Devices       0.46      0.11      0.17       398

                 

In [23]:
y_pred_elasticnet_logreg_test = elasticnet_logreg_model.predict(X_test)
print("Elasticnet Logistic Regression test:")
print(classification_report(y_test, y_pred_elasticnet_logreg_test, zero_division=0, target_names=val_dataset.pathologies))

Elasticnet Logistic Regression test:
                            precision    recall  f1-score   support

               Atelectasis       0.51      0.16      0.24       855
              Cardiomegaly       0.60      0.19      0.29       761
             Consolidation       0.00      0.00      0.00       166
                     Edema       0.49      0.14      0.22       229
Enlarged Cardiomediastinum       0.00      0.00      0.00       139
                  Fracture       0.00      0.00      0.00       184
               Lung Lesion       0.00      0.00      0.00       271
              Lung Opacity       0.55      0.08      0.13      1120
                  Effusion       0.66      0.51      0.57       742
             Pleural Other       0.00      0.00      0.00        81
                 Pneumonia       0.70      0.05      0.10       431
              Pneumothorax       0.00      0.00      0.00       105
           Support Devices       0.47      0.04      0.08       398

         